In [1]:
import pandas as pd
import numpy as np

loans = pd.read_csv('lending-club-data.csv')
loans.head()

/home/qiangwennorge/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [2]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop(['bad_loans'], axis = 1)

In [3]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [4]:
import json

train_idx = json.loads(open('module-5-assignment-2-train-idx.json').read())
test_idx = json.loads(open('module-5-assignment-2-test-idx.json').read())

train_data = loans.iloc[train_idx]
test_data = loans.iloc[test_idx]
train_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loans
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1,-1
6,1071795,1306957,5600,5600,5600,60 months,21.28,152.39,F,F2,...,1.0,1.0,1.0,0,4.57170,20161201T000000,1,1,1,-1
7,1071570,1306721,5375,5375,5350,60 months,12.69,121.45,B,B5,...,1.0,1.0,1.0,1,9.71600,20161201T000000,1,1,1,-1
10,1064687,1298717,9000,9000,9000,36 months,13.49,305.38,C,C1,...,1.0,1.0,1.0,1,12.21520,20141201T000000,1,1,1,-1
12,1069057,1303503,10000,10000,10000,36 months,10.65,325.74,B,B2,...,1.0,1.0,1.0,0,3.90888,20141201T000000,1,1,1,-1


### Subsample dataset to make sure classes are balanced

Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. You should have code analogous to

In [5]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(frac = percentage, random_state = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print("Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data)))
print("Percentage of risky loans                :", len(risky_loans) / float(len(loans_data)))
print("Total number of loans in our new dataset :", len(loans_data))

Percentage of safe loans                 : 0.5
Percentage of risky loans                : 0.5
Total number of loans in our new dataset : 46300


### Transform categorical data into binary features

In this assignment, we will implement binary decision trees (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features.

For instance, the home_ownership feature represents the home ownership status of the loanee, which is either own, mortgage or rent. For example, if a data point has the feature

In [6]:
grade_val = list(loans_data['grade'].unique())
len(grade_val)

7

In [7]:
term_val = list(loans_data['term'].unique())
len(term_val)

2

In [8]:
home_ownership_val = list(loans_data['home_ownership'].unique())
len(home_ownership_val)

4

In [9]:
emp_length_val = list(loans_data['emp_length'].unique())
len(emp_length_val)

12

In [39]:
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = pd.DataFrame(list(loans_data_one_hot_encoded)).fillna(0)   
    

    loans_data_new = loans_data_unpacked
    #loans_data.add_columns(loans_data_unpacked)

#loans_data_unpacked
loans_data

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loans
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1,-1
6,1071795,1306957,5600,5600,5600,60 months,21.28,152.39,F,F2,...,1.0,1.0,1.0,0,4.57170,20161201T000000,1,1,1,-1
7,1071570,1306721,5375,5375,5350,60 months,12.69,121.45,B,B5,...,1.0,1.0,1.0,1,9.71600,20161201T000000,1,1,1,-1
10,1064687,1298717,9000,9000,9000,36 months,13.49,305.38,C,C1,...,1.0,1.0,1.0,1,12.21520,20141201T000000,1,1,1,-1
12,1069057,1303503,10000,10000,10000,36 months,10.65,325.74,B,B2,...,1.0,1.0,1.0,0,3.90888,20141201T000000,1,1,1,-1
18,1039153,1269083,21000,21000,21000,36 months,12.42,701.73,B,B4,...,1.0,1.0,1.0,0,8.01977,20141201T000000,1,1,1,-1
21,1069559,1304634,6000,6000,6000,36 months,11.71,198.46,B,B3,...,1.0,1.0,1.0,0,3.13358,20141201T000000,1,1,1,-1
23,1069800,1304679,15000,15000,8725,36 months,14.27,514.64,C,C2,...,1.0,1.0,1.0,0,10.29280,20141201T000000,1,1,1,-1
24,1069657,1304764,5000,5000,5000,60 months,16.77,123.65,D,D2,...,0.0,1.0,1.0,0,2.96736,20161201T000000,0,1,1,-1
41,1069465,1304521,5000,5000,5000,36 months,8.90,158.77,A,A5,...,1.0,1.0,1.0,0,1.90524,20141201T000000,1,1,1,-1
